# Introduction Problem

 Imagine a 2D maze represented by a grid of cells.  Some cells are walls (obstacles) and others are open spaces.  We want to find the shortest path from a designated Start cell to a designated Goal cell while navigating around the walls.

In [7]:
class Node:
  def __init__(self, position, parent=None, g=0, h=0):
    self.position = position
    self.parent = parent
    self.g = g  # Cost from start to current node
    self.h = h  # Heuristic estimate of cost to goal
    self.f = self.g + self.h  # Total cost (f(n))

def heuristic(current, goal):
  # Manhattan distance heuristic (estimate to reach goal)
  return abs(current.position[0] - goal.position[0]) + abs(current.position[1] - goal.position[1])

def a_star(maze, start, goal):
  open_set = []  # Nodes to be explored (priority queue)
  closed_set = set()  # Explored nodes
  start_node = Node(start)
  open_set.append(start_node)

  while open_set:
    current = min(open_set, key=lambda x: x.f)  # Get node with lowest f(n)
    open_set.remove(current)
    closed_set.add(current.position)

    if current.position == goal:
      path = []
      while current:
        path.append(current.position)
        current = current.parent
      return path[::-1]  # Reverse path for start to goal

    for neighbor in get_neighbors(current, maze):
      if neighbor in closed_set:
        continue
      tentative_g = current.g + 1  # Cost of moving to neighbor

      if neighbor not in (node for node in open_set if node.position == neighbor.position):
        new_node = Node(neighbor, current, tentative_g)
      else:
        new_node = next(node for node in open_set if node.position == neighbor.position)
      
      if tentative_g < new_node.g:
        new_node.parent = current
        new_node.g = tentative_g
        new_node.h = heuristic(new_node, goal)
        new_node.f = new_node.g + new_node.h
        open_set.append(new_node)

  return None  # No path found

# Helper functions to define maze structure, get neighbors, etc. (replace with your maze representation)
def get_neighbors(node, maze):
  # Implement logic to get valid neighboring cells within the maze boundaries
  # based on node position and considering walls
  pass  

def maze_representation():
  # Define your maze layout here using a 2D list or other suitable data structure
  # with characters representing walls and open spaces 
  pass

# Example Usage
maze = maze_representation()

def get_neighbors(node, maze):
  # Implement logic to get valid neighboring cells within the maze boundaries
  # based on node position and considering walls
  x, y = node.position
  neighbors = [(nx, ny) for nx, ny in [(x-1, y), (x+1, y), (x, y-1), (x, y+1)] if 0 <= nx < len(maze) and 0 <= ny < len(maze[0]) and maze[nx][ny] != 1]
  return neighbors  

def maze_representation():
  # Define your maze layout here using a 2D list or other suitable data structure
  # with characters representing walls and open spaces 
  return [
        [0, 0, 0, 0],
        [0, 1, 1, 0],
        [0, 0, 0, 1],
        [1, 1, 0, 0]
    ]


### Visualization

In [13]:
import tkinter as tk
import PIL
from PIL import Image, ImageTk
def visualize_maze_tkinter(maze, path=None, start=None, goal=None):
    # Create a new Tkinter window
    window = tk.Tk()

    # Set the window size to 640x860
    window.geometry("640x640")

    # Calculate the size of each cell
    cell_size = 100
    width = len(maze[0]) * cell_size
    height = len(maze) * cell_size

    # Create a canvas to draw the maze
    canvas = tk.Canvas(window, width=width, height=height)
    canvas.pack()

    # "Load the robot image
    robot_image = Image.open("robot_icon.png")
    robot_image = robot_image.resize((cell_size, cell_size), Image.LANCZOS)  # Resize the image
    robot_icon = ImageTk.PhotoImage(robot_image)
    
    # Draw the maze
    for i, row in enumerate(maze):
        for j, cell in enumerate(row):
            x1 = j * cell_size
            y1 = i * cell_size
            x2 = x1 + cell_size
            y2 = y1 + cell_size
            color = 'white' if cell == 0 else 'black'
            canvas.create_rectangle(x1, y1, x2, y2, fill=color, outline="gray")  # Added outline for grid

    # Draw the start and goal positions
    if start:
        x1 = start[1] * cell_size
        y1 = start[0] * cell_size
        x2 = x1 + cell_size
        y2 = y1 + cell_size
        canvas.create_rectangle(x1, y1, x2, y2, fill='green')  # Start is green

    if goal:
        x1 = goal[1] * cell_size
        y1 = goal[0] * cell_size
        x2 = x1 + cell_size
        y2 = y1 + cell_size
        canvas.create_rectangle(x1, y1, x2, y2, fill='blue')  # Goal is blue

    # Draw the robot image at the start position
    if start:
        x = start[1] * cell_size + cell_size // 2
        y = start[0] * cell_size + cell_size // 2
        canvas.create_image(x, y, image = robot_icon)

    # Draw the path
    if path:
        for i in range(len(path)):
            x1 = path[i][1] * cell_size
            y1 = path[i][0] * cell_size
            x2 = x1 + cell_size
            y2 = y1 + cell_size
            canvas.create_rectangle(x1, y1, x2, y2, fill='red')

            # Determine direction and rotate image accordingly
            if i > 0:  # Skip for the first position
                dx = path[i][1] - path[i-1][1]
                dy = path[i][0] - path[i-1][0]
                if dx == 1:  # Moving right
                    angle = 0
                elif dx == -1:  # Moving left
                    angle = 180
                elif dy == 1:  # Moving down
                    angle = 90
                else:  # Moving up
                    angle = -90

                # Rotate and draw the robot image
                rotated_image = robot_image.rotate(angle)
                robot_icon = ImageTk.PhotoImage(rotated_image)
                x = path[i][1] * cell_size + cell_size // 2
                y = path[i][0] * cell_size + cell_size // 2
                canvas.create_image(x, y, image=robot_icon)


    # Start the Tkinter event loop
    window.mainloop()

# Example usage
maze = maze_representation()
start = (0, 0)
goal = (3, 3)
path = a_star(maze, start, goal)
visualize_maze_tkinter(maze, path, start, goal)